In [1]:
import pandas as pd
import re 
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [7]:
df = pd.read_csv('final_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(axis=0, inplace=True)

In [8]:
df.head()

,vid,title,description,category
0,ehmsJLZlCZ0,\r\n Ep 1| Travelling through North East In...,"The journey to Arunachal, North East India beg...",travel
1,yvn79Rv0F48,\r\n Backpacking In Meghalaya | NorthEast I...,"In this video I explored North East India, sta...",travel
2,i9E_Blai8vk,\r\n Welcome to Bali | Travel Vlog | Prisci...,I had the chance to fly out to Bali with my wh...,travel
3,Qvg1EW2FFmU,\r\n Travel Vlog | Weekend In Pondicherry |...,Equipment I use for filming: http://geni.us/vK...,travel
4,_lcOX55Ef70,\r\n Solo in Paro Bhutan | Tiger's Nest vis...,Here’s presenting the first part of the Bhutan...,travel


In [9]:
corpus = []
for title in df['title']:
  review = re.sub('[^a-zA-Z]', ' ', title)
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [10]:
i=0
corpus1 = []
for description in df['description']:
  review = re.sub('[^a-zA-Z]', ' ', description)
  i+=1
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus1.append(review)

KeyboardInterrupt: 

In [11]:
df_title = pd.DataFrame({'title':corpus})
#df_description = pd.DataFrame({'description':corpus1})

df_category = df[['category']]

from sklearn.preprocessing import LabelEncoder

In [13]:
df_title.head()

,title
0,ep travel north east india arunach journey beg...
1,backpack meghalaya northeast india trip sohra ...
2,welcom bali travel vlog priscilla lee
3,travel vlog weekend pondicherri kritika goel
4,solo paro bhutan tiger nest visit bhutan trave...


In [15]:
df_category.head()

,category
0,travel
1,travel
2,travel
3,travel
4,travel


In [19]:
dfcategory = df_category.apply(LabelEncoder().fit_transform)

#df_new = pd.concat([df[['vid']], df_title, df_description, dfcategory], axis=1, join_axes= [df[['vid']].index])
df_new = pd.concat([df[['vid']], df_title, dfcategory], axis=1)
df_new = df_new.reindex(df.index)

In [21]:
df_new.head()

,vid,title,category
0,ehmsJLZlCZ0,ep travel north east india arunach journey beg...,5.0
1,yvn79Rv0F48,backpack meghalaya northeast india trip sohra ...,5.0
2,i9E_Blai8vk,welcom bali travel vlog priscilla lee,5.0
3,Qvg1EW2FFmU,travel vlog weekend pondicherri kritika goel,5.0
4,_lcOX55Ef70,solo paro bhutan tiger nest visit bhutan trave...,5.0


In [29]:
from sklearn.feature_extraction.text import CountVectorizer   
cv = CountVectorizer(max_features = 1500) 
#X = cv.fit_transform(corpus, corpus1).toarray() 
X = cv.fit_transform(corpus).toarray() 
y = df_new.iloc[:, 2].values #select category column for vectorization

In [34]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy')
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=1000)

In [35]:
y_pred = classifier.predict(X_test)
classifier.score(X_test, y_test)

0.9809220985691574

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97       132
         1.0       1.00      0.95      0.97        79
         2.0       0.99      0.99      0.99        72
         3.0       1.00      0.98      0.99       114
         4.0       0.99      0.99      0.99       107
         5.0       0.98      0.97      0.98       125

    accuracy                           0.98       629
   macro avg       0.98      0.98      0.98       629
weighted avg       0.98      0.98      0.98       629



In [37]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[132,   0,   0,   0,   0,   0],
       [  1,  75,   0,   0,   1,   2],
       [  1,   0,  71,   0,   0,   0],
       [  2,   0,   0, 112,   0,   0],
       [  0,   0,   1,   0, 106,   0],
       [  4,   0,   0,   0,   0, 121]], dtype=int64)